# PSyclone tutorial: Use of Fparser2 in PSyclone

This example shows how PSyclone reads existing Fortran code using a parser called fparser2.

First let's specify a simple Fortran code in a Python string. PSyclone typically reads codes from files but it is easier to demonstrate what is happening using a string.

In [ ]:
code = '''program test
  implicit none
  integer, parameter :: ni=10, nj=10, nk=10
  real, dimension(ni,nj,nk) :: a,b
  integer :: i,j,k
  do k=1,nk
    do j=1,nj
      do i=1,ni
        a(i,j,k) = b(i,j,k)
      end do
    end do
  end do
end program test'''

PSyclone parses this code using a Fortran parser called fparser2 (see https://fparser.readthedocs.io/en/stable/ if you are interested). Notice that we choose the Fortran2003 standard (`std=f2003`) below. It is also possible to choose Fortran 2008 (std=`f2008`), however newer standards are not yet available in fparser2.

In [ ]:
from fparser.common.readfortran import FortranStringReader
reader = FortranStringReader(code)
from fparser.two.parser import ParserFactory
parser = ParserFactory().create(std="f2003")
parse_tree = parser(reader)

The parsed code (stored in the `parse_tree` variable) can be printed out. As expected, the same code that we defined in the original code is output. The only way to know that something has happened is that the formatting and case has changed.

In [ ]:
print(str(parse_tree))

The parser has created a tree of nodes which conform to the rules that specify the Fortran 2003 language. Next we print out this tree as text to see all of the nodes that have been created. The names of these nodes correspond to the rules that specify the Fortran 2003 language. If you are interested you can see these rules here (https://wg5-fortran.org/N1601-N1650/N1601.pdf).

In [ ]:
print(repr(parse_tree))

So, why do we use fparser2 in PSyclone? The main reason is that it makes it very easy to extract information about the code in a robust way. Originally the code was stored as a string, now it is stored in a structured way. For example, if we want to get access to all the assignment statements ...

In [ ]:
from fparser.two.utils import walk
from fparser.two.Fortran2003 import Assignment_Stmt
for node in walk(parse_tree, Assignment_Stmt):
    print(node)

It is also possible to modify the parse tree, although it is currently a little cumbersome due to the use of tuples which are immutable. Improving this in fparser2 is work in progress. In the example below we change the values of the `ni`, `nj` and `nk` parameters.

In [ ]:
from fparser.two.utils import walk
from fparser.two.Fortran2003 import Int_Literal_Constant
for node in walk(parse_tree, Int_Literal_Constant):
    children = list(node.items)
    children[0] = "20"
    node.items = tuple(children)
print(parse_tree)

If you would like to try something yourself why not try to print out all of the  `do` statements. Hint: follow the approach taken for printing assignment statements.

Congratulations. You've completed the fparser2 section of the tutorial.
Back to the [Introduction section](../introduction.ipynb).